## CS102-4 - Further Computing

Prof. Götz Pfeiffer<br>
School of Mathematics, Statistics and Applied Mathematics<br>
NUI Galway

### 1. Aspects of Scientific Computing

# Week 4: Computation on `NumPy` Arrays: Universal Functions and Broadcasting.

* `numpy` provides efficient **storage** for homogeneous
  multidimensional data.
* `numpy` also provides efficient **operations** on such data.
* Some of these **vectorized** operations are implemented as 
  **universal functions**, or **UFuncs** for short.
* **Broadcasting** is a set of rules for applying binary UFuncs on arrays of different sizes.

In [ ]:
import numpy as np

## `python`'s  Loops  are slow

* Due to the dynamic, interpreted nature of the language
  certain sequences of operations cannot be compiled into efficient 
  machine code as in languages like `C` and `Fortran`.

* In situations where many small operations are being repeated. for instance when looping over an array,
`python` has to repeat certain checks for each element in the array.

* Some of those checks become unnecessary, when it is known in advance
  that all elements in the array have the same type.

* For example, let's compute the reciprocals of an array of values. 

In [ ]:
def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
print(values)
compute_reciprocals(values)

* Note how the code makes use of the efficient `numpy` data
  structures and how it avoids the need for resizing
  the `output` array.
  
* Now let's time the operation for a bigger array:

In [ ]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

* It takes more than a second to compute these 1 million inverses
  and to store the result!

* For today's standards, that's a long time ...

## Introducing UFuncs

* For many types of operations, `NumPy` provides a **vectorized** 
  alternative.

* This vectorized approach is designed to push the loop into the compiled layer that underlies NumPy, leading to much faster execution.

* Use a **UFunc** like this:  simply perform the operation on the array.

In [ ]:
print(compute_reciprocals(values))
print(1.0 / values)

* time the UFunc:

In [ ]:
%timeit 1.0 / big_array

* `ms` here stands for millisecond.  That is one 1000th of a second!

* UFuncs are extremely flexible:

In [ ]:
np.arange(5) / np.arange(1, 6)

* UFunc operations can also act on multi-dimensional arrays:

In [ ]:
x = np.arange(9).reshape((3, 3))
2 ** x

* UFuncs are nearly always more efficient than `Python` loops, especially as the arrays grow in size.

## Exploring NumPy's UFuncs

* UFuncs exist in two flavors: **unary** ufuncs, which operate on a single input, and **binary** ufuncs, which operate on two inputs.

### Array arithmetic

* `NumPy`'s UFuncs make use of `Python`'s native arithmetic operators,
e.g., for addition, subtraction, multiplication, and division (all binary).

In [ ]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

* There is also a unary ufunc for negation, and a ``**`` operator for exponentiation, and a ``%`` operator for modulus:

In [ ]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

* UFuncs can be strung together, and the standard order of operations is respected:

In [ ]:
-(0.5*x + 1) ** 2

* Each of these arithmetic operations are simply convenient wrappers around specific functions built into `NumPy`; for example, the `+` operator is a wrapper for the `add` function:

In [ ]:
np.add(x, 2)

* The following table lists the arithmetic operators implemented in NumPy:

| Operator	    | Equivalent ufunc    | Description                           |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |Addition (e.g., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |Subtraction (e.g., ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |Unary negation (e.g., ``-2``)          |
|``*``          |``np.multiply``      |Multiplication (e.g., ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |Division (e.g., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |Floor division (e.g., ``3 // 2 = 1``)  |
|``**``         |``np.power``         |Exponentiation (e.g., ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |Modulus/remainder (e.g., ``9 % 4 = 1``)|

* Additionally there are Boolean/bitwise operators ... 

### Absolute value

In [ ]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

In [ ]:
np.absolute(x)

In [ ]:
np.abs(x)

In [ ]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

### Trigonometric functions

In [ ]:
theta = np.linspace(0, np.pi, 3)

* Compute some trigonometric functions on these values:

In [ ]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

* The values are computed to within machine precision: some zero 
values do not always hit exactly zero.

In [ ]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

### Exponents and Logarithms

In [ ]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

* The `np.log` gives the natural logarithm,
`np.log2` the the base-2 logarithm 
and `np.log10` the base-10 logarithm.

In [ ]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

* Some specialized versions that are useful for maintaining precision with very small input:

In [ ]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

### Specialized ufuncs

* The submodule `scipy.special` is a source for more specialized and obscure UFuncs.

In [ ]:
from scipy import special

In [ ]:
# Gamma functions (generalized factorials) and related functions
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

In [ ]:
# Error function (integral of Gaussian)
# its complement, and its inverse
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

## Advanced UFunc Features

### Specifying output

* For large calculations, it is sometimes useful 
  or even necessary
  to specify the array where the result
  of the calculation will be stored.

* For all UFuncs, this can be done with the
  `out` argument of the function:

In [ ]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

* This also works with array views. For example, to write the results of a computation to every other element of a specified array:

In [ ]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

* Note that the code `y[::2] = 2 ** x` results in the creation of a temporary array to hold the results of `2 ** x`, followed by a second operation copying those values into the ``y`` array.

### Aggregates

* Binary UFuncs can be used to compute **aggregates** directly from the object.

* A **reduce** repeatedly applies a given operation to the elements of an array until only a single result remains.

* To reduce an array with a particular operation, use the `reduce`
  method of the corresponding UFunc.

* For example, calling `reduce` on the `add` UFunc returns
  the sum of all elements in the array:

In [ ]:
x = np.arange(1, 6)
np.add.reduce(x)

* Similarly, calling `reduce` on the `multiply` ufunc results in the 
  product of all array elements:

In [ ]:
np.multiply.reduce(x)

* To store and return all the intermediate results of the computation, 
  use `accumulate` instead:

In [ ]:
np.add.accumulate(x)

In [ ]:
np.multiply.accumulate(x)

* In fact, for these particular cases, there are dedicated `NumPy` functions to compute the results (``np.sum``, ``np.prod``, ``np.cumsum``, ``np.cumprod``) ...

### Outer products

* A UFunc can compute the output of all pairs of two different inputs using the ``outer`` method.

* With this one can create things like create a multiplication table
  in one line: 

In [ ]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

In [ ]:
np.multiply(x, x)

* UFuncs can operate between arrays of **different sizes** and **shapes**, via a convention known as **broadcasting** ...

## Broadcasting

* Broadcasting is a set of rules for applying binary UFuncs on arrays of different sizes.

* For arrays of the same size, binary operations are performed on an element-by-element basis:

In [ ]:
a = np.array([0, 1, 2])
b = np.array([5, 5, 5])
a + b

* The same result is obtained by adding the **scalar** $5$
  (a zero-dimensional array) to the array `a`.

In [ ]:
a + 5

* Think of this as an operation that first stretches or duplicates the value `5` into the array `[5, 5, 5]`, and then adds the two arrays
of the same shape.

* Inside `NumPy`, this duplication of values does not actually take place, but it helps to describe how broadcasting works.

* The concept extends to arrays of higher dimension, e.g, the sum
  of a one-dimensional array (a matrix) and a two-dimensional array
  (a vector):

In [ ]:
M = np.ones((3, 3))
M

In [ ]:
M + a

* Here the one-dimensional array `a` is stretched, or broadcast across the second dimension in order to match the shape of `M`.

* More complicated cases can involve broadcasting of both arrays
  in order to yield a common shape.
  
* The sum of a row vector and a column vector, for example.

In [ ]:
a = np.arange(3)
b = np.arange(3)[:, np.newaxis]
print(a)
print(b)
a + b

## Rules of Broadcasting

Broadcasting in `NumPy` follows a **strict set of rules** to determine the interaction between the two arrays:

- **Rule 1:** If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is **padded** with ones on its leading (left) side.
- **Rule 2:** If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.
- **Rule 3:** If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

To make these rules clear, let's consider a few examples in detail.

### Broadcasting example 1

Let's look at adding a two-dimensional array to a one-dimensional array:

In [ ]:
M = np.ones((2, 3))
a = np.arange(3)

The shapes of the arrays are

- ``M.shape = (2, 3)``
- ``a.shape = (3,)``

Rule 1: the array ``a`` has fewer dimensions, so we pad it on the left with ones:

- ``M.shape -> (2, 3)``
- ``a.shape -> (1, 3)``

Rule 2: the first dimension disagrees, so we stretch this dimension to match:

- ``M.shape -> (2, 3)``
- ``a.shape -> (2, 3)``

The shapes match, and the final shape will be ``(2, 3)``:

In [ ]:
M + a

### Broadcasting example 2

* Let's take a look at an example where both arrays need to be broadcast:

In [ ]:
a = np.arange(3).reshape((3, 1))
b = np.arange(3)

Start by writing out the shape of the arrays:

- ``a.shape = (3, 1)``
- ``b.shape = (3,)``

Rule 1: pad the shape of ``b`` with ones:

- ``a.shape -> (3, 1)``
- ``b.shape -> (1, 3)``

Rule 2: upgrade **each** of these ones to match the corresponding size of the other array:

- ``a.shape -> (3, 3)``
- ``b.shape -> (3, 3)``

Because the result matches, these shapes are compatible.

In [ ]:
a + b

### Broadcasting example 3

* Finally, let's take a look at an example in which the two arrays are not compatible:

In [ ]:
M = np.ones((3, 2))
a = np.arange(3)

The matrix ``M`` is transposed, compared to the first example.
The shapes of the arrays now are

- ``M.shape = (3, 2)``
- ``a.shape = (3,)``

Rule 1: pad the shape of ``a`` with ones:

- ``M.shape -> (3, 2)``
- ``a.shape -> (1, 3)``

Rule 2: the first dimension of ``a`` is stretched to match that of ``M``:

- ``M.shape -> (3, 2)``
- ``a.shape -> (3, 3)``

Rule 3: the final shapes do not match, so these two arrays are incompatible.

* The command
```python
M + a
```
would now result in an error ...

* If right-side padding is really needed, one can always reshape the array explicitly:

In [ ]:
a[:, np.newaxis].shape

In [ ]:
M + a[:, np.newaxis]

* The examples have focused on the ``+`` operator.
* These broadcasting rules apply to **any** binary ``ufunc``.
* For example, here is the ``logaddexp(a, b)`` function, which computes ``log(exp(a) + exp(b))`` with more precision than the naive approach:

In [ ]:
np.logaddexp(M, a[:, np.newaxis])

## Broadcasting in Practice

* Broadcasting operations will be extremely useful as we move on.

### Centering an array

* Imagine you have an array of 10 observations, each of which consists of 3 values.

* Store this data in a $10 \times 3$ array:

In [ ]:
X = np.random.random((10, 3))

* Compute the mean of each feature using the ``mean`` aggregate across the first dimension:

In [ ]:
Xmean = X.mean(0)
Xmean

* Center the ``X`` array by subtracting the mean (via broadcasting):

In [ ]:
X_centered = X - Xmean

* Check that the centered array has near zero mean:

In [ ]:
X_centered.mean(0)

* To within machine precision, the mean is now zero.

### Plotting a two-dimensional function

* Broadcasting can be used to compute a function $z = f(x, y)$ across a grid:

In [ ]:
# x and y have 50 steps from 0 to 5
x = np.linspace(0, 5, 50)
y = np.linspace(0, 5, 50)[:, np.newaxis]

z = np.sin(x) ** 10 + np.cos(10 + y * x) * np.cos(x)

* Use `Matplotlib` to plot this two-dimensional array of values:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(z, origin='lower', extent=[0, 5, 0, 5],
           cmap='viridis')
plt.colorbar();

* ...

## References

* UFuncs [[doc]](https://docs.scipy.org/doc/numpy/reference/ufuncs.html)

## Exercises